<a href="https://colab.research.google.com/github/crossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Importações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
import requests
import warnings
from io import StringIO
from google.colab import drive  # Importe o módulo drive

# --- Configurações e Constantes ---
warnings.filterwarnings('ignore')

DEFAULT_PALETTE = "viridis"
FIGSIZE = (10, 6)

# Estilo dos gráficos
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black',
    'axes.facecolor': 'black',
    'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan',
    'xtick.color': 'white',
    'ytick.color': 'white',
    'text.color': 'white',
    'grid.color': 'gray',
    'grid.linestyle': '--',
    'legend.facecolor': 'black',
    'legend.edgecolor': 'white',
    'figure.titlesize': 18,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})

# --- Funções Auxiliares ---

def mount_google_drive():
    """Monta o Google Drive no ambiente Colab."""
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    """Retorna o caminho completo no Google Drive."""
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory_exists_on_drive(relative_path):
    """Garante que o diretório exista no Google Drive."""
    drive_path = get_drive_path(relative_path)
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"Diretório criado no Google Drive: {drive_path}")
    return drive_path


def save_fig(fig, filename, drive_folder_path):
    """Salva o gráfico no Google Drive."""
    try:
        filepath = os.path.join(drive_folder_path, filename)
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Gráfico salvo no Google Drive: {filepath}")
    except Exception as e:
        print(f"Erro ao salvar o gráfico '{filename}': {e}")
    finally:
        plt.close(fig)


def create_figure():
    """Cria uma nova figura."""
    return plt.figure(figsize=FIGSIZE)


def generate_statistical_summary(df, filename="sumario.txt", drive_path="/content/drive/MyDrive"):
    """Gera um sumário estatístico e salva no Google Drive."""
    try:
        summary = df.describe(include='all').to_string()
        filepath = os.path.join(drive_path, filename)
        with open(filepath, 'w') as f:
            f.write(summary)
        print(f"Sumário estatístico salvo em: {filepath}")
    except Exception as e:
        print(f"Erro ao gerar sumário estatístico: {e}")


# --- Funções de Visualização ---
def generate_correlation_heatmap(df, title, filename, drive_folder_path):
    """Gera um heatmap de correlação."""
    try:
        df_numeric = df.select_dtypes(include=np.number)
        if df_numeric.empty:
            print("Nenhum dado numérico disponível para gerar o heatmap.")
            return
        fig = create_figure()
        sns.heatmap(df_numeric.corr(), annot=True, cmap=DEFAULT_PALETTE, fmt=".2f", linewidths=.5)
        plt.title(title)
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar heatmap: {e}")


def generate_scatterplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Gera um scatterplot."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{y_col}' não encontradas no DataFrame.")
            return
        fig = create_figure()
        sns.scatterplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE, alpha=0.7)
        plt.title(title)
        plt.xticks(rotation=45)
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar scatterplot: {e}")


def generate_histogram(df, col, title, filename, drive_folder_path, bins=20, kde=True):
    """Gera um histograma."""
    try:
        if col not in df.columns:
            print(f"Coluna '{col}' não encontrada no DataFrame.")
            return
        fig = create_figure()
        sns.histplot(df[col], kde=kde, bins=bins, color='skyblue')
        plt.title(title)
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar histograma: {e}")


def generate_boxplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Gera um boxplot."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{y_col}' não encontradas no DataFrame.")
            return
        fig = create_figure()
        sns.boxplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xticks(rotation=45)
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar boxplot: {e}")


def generate_violinplot(df, x_col, y_col, title, filename, drive_folder_path, hue=None):
    """Gera um violinplot."""
    try:
        if x_col not in df.columns or y_col not in df.columns:
            print(f"Colunas '{x_col}' ou '{y_col}' não encontradas no DataFrame.")
            return
        fig = create_figure()
        sns.violinplot(x=x_col, y=y_col, data=df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title)
        plt.xticks(rotation=45)
        save_fig(fig, filename, drive_folder_path)
    except Exception as e:
        print(f"Erro ao gerar violinplot: {e}")


def generate_pca_plot(df, features, title, filename, drive_folder_path, hue=None):
    """Gera um gráfico de PCA."""
    try:
        df_pca = df[features].select_dtypes(include=np.number).dropna()

        if df_pca.shape[1] < 2:
            print("Não há colunas numéricas suficientes para realizar a PCA.")
            return

        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(df_pca)
        pca = PCA(n_components=2)
        principal_components = pca.fit_transform(scaled_data)
        pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

        if hue and hue in df.columns:
            pca_df[hue] = df.loc[df_pca.index, hue].values

        fig = create_figure()
        sns.scatterplot(x='PC1', y='PC2', data=pca_df, hue=hue, palette=DEFAULT_PALETTE)
        plt.title(title)
        save_fig(fig, filename, drive_folder_path)

    except Exception as e:
        print(f"Erro ao gerar gráfico de PCA: {e}")


# --- Função Principal (main) ---
if __name__ == "__main__":
    try:
        # Montar o Google Drive
        mount_google_drive()

        # Criar/Garantir a existência da pasta 'graficos' no Google Drive
        graficos_drive_path = ensure_directory_exists_on_drive('graficos')

        # Baixar os dados do URL
        url = "https://docs.google.com/spreadsheets/d/1e8otjghJ3PGvUNnQZevu6bZ0ktjBfWWUXvkRm61_y_8/export?format=csv"
        response = requests.get(url)
        response.raise_for_status()

        # Carregar os dados em um DataFrame
        df = pd.read_csv(StringIO(response.text))

        # Pré-processamento
        df.dropna(axis=1, how='all', inplace=True)
        for col in df.select_dtypes(include=np.number):
            df[col] = df[col].fillna(df[col].median())
        if 'Regiao' in df:
            df['Regiao_encoded'] = LabelEncoder().fit_transform(df['Regiao'])
        if 'Setor Mais Atacado' in df:
            df['Setor_encoded'] = LabelEncoder().fit_transform(df['Setor Mais Atacado'])

        # --- Análises e Visualizações ---
        generate_correlation_heatmap(df, "1. Matriz de Correlação Expandida", "1_heatmap.png", graficos_drive_path)
        generate_scatterplot(df, "IDH", "Ataques_Ciberneticos_num", "2. IDH vs Ataques", "2_scatter_idh_ataques.png", graficos_drive_path, hue="Regiao_encoded")
        generate_scatterplot(df, "Indice_de_Gini", "Ataques_Ciberneticos_num", "3. Gini vs Ataques", "3_scatter_gini_ataques.png", graficos_drive_path, hue="Regiao_encoded")
        generate_histogram(df, "Ataques_Ciberneticos_num", "4. Histograma: Ataques", "4_hist_ataques.png", graficos_drive_path)
        generate_boxplot(df, "Regiao", "Ataques_Ciberneticos_num", "5. Boxplot: Ataques por Região", "5_boxplot_ataques_regiao.png", graficos_drive_path)
        generate_violinplot(df, "Regiao", "Investimento_Ciberseguranca_per_capita_R_", "6. Violinplot: Investimento por Região", "6_violin_investimento_regiao.png", graficos_drive_path)
        generate_scatterplot(df, "Escolaridade_Superior__", "Impacto_Financeiro_Ataques_R_Milhoes", "7. Escolaridade vs Impacto", "7_scatter_escolaridade_impacto.png", graficos_drive_path, hue="Regiao_encoded")
        generate_histogram(df, "Investimento_Ciberseguranca_per_capita_R_", "8. Histograma: Investimento", "8_hist_investimento.png", graficos_drive_path)
        generate_boxplot(df, "Regiao", "Impacto_Financeiro_Ataques_R_Milhoes", "9. Boxplot: Impacto por Região", "9_boxplot_impacto_regiao.png", graficos_drive_path)
        generate_pca_plot(df, ['IDH', 'Indice_de_Gini', 'Escolaridade_Superior__', 'Investimento_Ciberseguranca_per_capita_R_'], "10. PCA", "10_pca.png", graficos_drive_path, hue="Regiao_encoded")

        # --- Sumário Estatístico ---
        generate_statistical_summary(df, "sumario.txt")  # Salva na raiz do Google Drive (MyDrive)

        print("Análise completa, gráficos e sumário gerados!")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar o URL: {e}")
    except Exception as e:
        print(f"Erro geral no processamento: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/1_heatmap.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/2_scatter_idh_ataques.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/3_scatter_gini_ataques.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/4_hist_ataques.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/5_boxplot_ataques_regiao.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/6_violin_investimento_regiao.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/7_scatter_escolaridade_impacto.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/8_hist_investimento.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/9_boxplot_impacto_regiao.png
Gráfico salvo no Google Drive: /content/drive/MyDrive/graficos/10_pc